## TODO
Convert this notebook to a script and deploy to docker so it can be a cron job

In [3]:
response = requests.get('https://consumer-api.leafly.com/api/strains/v1/wedding-cake/reviews?skip=0&take=60&sort\\[0\\]\\[upvotes_count\\]=desc', headers=headers)


<Response [200]>

In [5]:
num_strains = response.json()['metadata']['totalCount']

In [6]:
num_strains

1780

In [37]:
all_review_data = []


In [ ]:
MAX_REVIEW_TAKE = 60
completed_strains = []

In [74]:
def get_strain_reviews(strain):
    if strain in completed_strains:
        return False
    review_data = []
    num_reviews_batches = math.ceil(get_review_num(strain)/60)
    for i in range(num_reviews_batches):
        time.sleep(.5)
        if i%20==0:
            print(f'{strain} : {i}/{num_reviews_batches}')
        try:
            data = get_review_batch(strain, i)        
            review_data.extend(iter(data))
        except Exception as e:
            print(f'failed to grab 60 at {60*i}')
            print(e)
    # https://stackoverflow.com/a/41801708
    reviews_df = pd.concat([pd.json_normalize(v, sep='_') for v in review_data])
    reviews_df.reset_index(drop=True, inplace=True)
    reviews_df.to_csv(f'../../data/raw/reviews/{strain}_reviews.csv')
    completed_strains.append(strain)

In [73]:
wedding_reviews = get_strain_reviews('wedding-cake')

wedding-cake : 0/30
wedding-cake : 20/30


In [ ]:
wedding_reviews

In [70]:
# https://stackoverflow.com/a/41801708
wedding_df = pd.concat([pd.json_normalize(v, sep='_') for v in wedding_reviews])
wedding_df.reset_index(drop=True, inplace=True)

In [71]:
wedding_df.to_csv('../../data/raw/reviews/weeding.csv')

In [66]:
get_review_num('wedding-cake')

1780

In [67]:
strains = pd.read_csv('../../data/raw/strains.csv')
strains

,id,averageRating,category,flowerImageSvg,name,nugImage,phenotype,reviewCount,shortDescriptionPlain,slug,...,terps_myrcene_score,terps_ocimene_name,terps_ocimene_description,terps_ocimene_score,terps_pinene_name,terps_pinene_description,terps_pinene_score,terps_terpinolene_name,terps_terpinolene_description,terps_terpinolene_score
0,233839,4.564078,Hybrid,https://public.leafly.com/strains/flowers/wedd...,Wedding Cake,https://leafly-public.s3-us-west-2.amazonaws.c...,Hybrid,2060,NaN,wedding-cake,...,0.355683,ocimene,NaN,0.008820,pinene,NaN,0.130472,terpinolene,NaN,0.027740
1,65591,4.639808,Hybrid,https://public.leafly.com/strains/flowers/orig...,GG4,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,4989,Original Glue (GG4) is a potent hybrid strain ...,original-glue,...,0.410000,ocimene,NaN,0.015000,pinene,NaN,0.118333,terpinolene,NaN,0.025000
2,504256,4.557951,Hybrid,https://public.leafly.com/strains/flowers/runt...,Runtz,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,742,NaN,runtz,...,0.062000,ocimene,NaN,0.017500,pinene,NaN,0.091542,terpinolene,NaN,0.024583
3,503179,4.618434,Indica,https://public.leafly.com/strains/flowers/ice-...,Ice Cream Cake,https://images.leafly.com/flower-images/defaul...,Indica,1009,NaN,ice-cream-cake,...,0.199178,ocimene,NaN,0.012209,pinene,NaN,0.126561,terpinolene,NaN,0.023545
4,174127,4.600363,Hybrid,https://public.leafly.com/strains/flowers/gela...,Gelato,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,2202,NaN,gelato,...,0.128459,ocimene,NaN,0.027078,pinene,NaN,0.127167,terpinolene,NaN,0.016123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6148,505850,0.000000,NaN,https://public.leafly.com/strains/flowers/bunn...,Bunny Hug,https://images.leafly.com/flower-images/defaul...,NaN,0,NaN,bunny-hug,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6149,505849,0.000000,NaN,https://public.leafly.com/strains/flowers/blue...,Blue Bean,https://images.leafly.com/flower-images/defaul...,NaN,0,NaN,blue-bean,...,2.499929,ocimene,NaN,0.317083,pinene,NaN,0.460348,terpinolene,NaN,0.000000
6150,505848,4.500000,NaN,https://public.leafly.com/strains/flowers/big-...,Big Lemon,https://images.leafly.com/flower-images/defaul...,NaN,2,NaN,big-lemon,...,0.226033,ocimene,NaN,0.000000,pinene,NaN,0.045100,terpinolene,NaN,0.022173
6151,505844,5.000000,NaN,https://public.leafly.com/strains/flowers/appl...,Appletini,https://images.leafly.com/flower-images/defaul...,NaN,1,NaN,appletini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
strains.slug.apply(lambda x :get_strain_reviews(x))

wedding-cake : 0/30
wedding-cake : 20/30
original-glue : 0/71
original-glue : 20/71
original-glue : 40/71
original-glue : 60/71
runtz : 0/12
ice-cream-cake : 0/15
gelato : 0/31
gelato : 20/31
blue-dream : 0/167
blue-dream : 20/167
blue-dream : 40/167
blue-dream : 60/167
blue-dream : 80/167
blue-dream : 100/167
blue-dream : 120/167
blue-dream : 140/167
blue-dream : 160/167
failed to grab 60 at 9960
'data'
sour-diesel : 0/103
sour-diesel : 20/103
sour-diesel : 40/103
sour-diesel : 60/103
sour-diesel : 80/103
sour-diesel : 100/103
do-si-dos : 0/16
apple-fritter : 0/5
gsc : 0/96
gsc : 20/96
gsc : 40/96
gsc : 60/96
gsc : 80/96
og-kush : 0/69
og-kush : 20/69
og-kush : 40/69
og-kush : 60/69
mac-1 : 0/6
purple-punch : 0/22
purple-punch : 20/22
mimosa : 0/14
jack-herer : 0/63
jack-herer : 20/63
jack-herer : 40/63
jack-herer : 60/63
granddaddy-purple : 0/70
granddaddy-purple : 20/70
granddaddy-purple : 40/70
granddaddy-purple : 60/70
sunset-sherbert : 0/23
sunset-sherbert : 20/23
pineapple-expre

KeyboardInterrupt: 

In [77]:
from multiprocessing import Pool


In [79]:
p = Pool()
records = p.map(get_strain_reviews, strains.slug)


Process SpawnPoolWorker-52:
Process SpawnPoolWorker-51:
Process SpawnPoolWorker-53:
Traceback (most recent call last):
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_strain_reviews' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/jachiike/opt/anaconda3/envs/base3/lib/python3.8/mul

KeyboardInterrupt: 

In [69]:
len(all_review_data)

9414

In [43]:
len(wedding_reviews)

1780

In [ ]:
def function_factory(n):
  return lambda a : a * n

mydoubler = function_factory(2)

print(mydoubler(11))

In [8]:
# https://stackoverflow.com/a/41801708
reviews_df = pd.concat([pd.json_normalize(v, sep='_') for v in all_review_data])
reviews_df.reset_index(drop=True, inplace=True)

In [10]:
reviews_df.head()

,id,averageRating,category,flowerImageSvg,name,nugImage,phenotype,reviewCount,shortDescriptionPlain,slug,...,terps_myrcene_score,terps_ocimene_name,terps_ocimene_description,terps_ocimene_score,terps_pinene_name,terps_pinene_description,terps_pinene_score,terps_terpinolene_name,terps_terpinolene_description,terps_terpinolene_score
0,233839,4.564078,Hybrid,https://public.leafly.com/strains/flowers/wedd...,Wedding Cake,https://leafly-public.s3-us-west-2.amazonaws.c...,Hybrid,2060,None,wedding-cake,...,0.355683,ocimene,None,0.008820,pinene,None,0.130472,terpinolene,None,0.027740
1,65591,4.639808,Hybrid,https://public.leafly.com/strains/flowers/orig...,GG4,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,4989,Original Glue (GG4) is a potent hybrid strain ...,original-glue,...,0.410000,ocimene,None,0.015000,pinene,None,0.118333,terpinolene,None,0.025000
2,504256,4.557951,Hybrid,https://public.leafly.com/strains/flowers/runt...,Runtz,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,742,None,runtz,...,0.062000,ocimene,None,0.017500,pinene,None,0.091542,terpinolene,None,0.024583
3,503179,4.618434,Indica,https://public.leafly.com/strains/flowers/ice-...,Ice Cream Cake,https://images.leafly.com/flower-images/defaul...,Indica,1009,None,ice-cream-cake,...,0.199178,ocimene,None,0.012209,pinene,None,0.126561,terpinolene,None,0.023545
4,174127,4.600363,Hybrid,https://public.leafly.com/strains/flowers/gela...,Gelato,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,2202,None,gelato,...,0.128459,ocimene,None,0.027078,pinene,None,0.127167,terpinolene,None,0.016123


In [9]:
strains_df.to_csv('../../data/raw/strains.csv',index=False)
